# Kalman Filters

## Theory

## Eule

## Helpers

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

ModuleNotFoundError: No module named 'pandas'

In [ ]:
class utilities:
  def __init__(self, dataset_url,row_start=0,row_end=100):
    self.fetch_data(dataset_url,row_start,row_end)

  def fetch_data(self, dataset_url,row_start=0,row_end=100):
      """
      Fetches a CSV from a URL and returns a pandas DataFrame.
      Handles errors during file reading and reports success metrics.
      """
      try:
          df = pd.read_csv(dataset_url)
          print("✅ Data loading successful!")
          self.full_df = df

          print(f"Total number of data rows loaded: {len(self.full_df)}")
          self.df = self.full_df.iloc[row_start:row_end]

          print(f"Total number of data rows selected: {len(self.df)}")
          print("\nHead of the loaded DataFrame:")
          print(self.df.head())


          self.df_gyro = self.df[['Gyro_x', 'Gyro_y', 'Gyro_z']]
          self.df_acc = self.df[['Acc_x', 'Acc_y', 'Acc_z']]
          self.df_gyro_acc = self.df[['Gyro_x', 'Gyro_y', 'Gyro_z', 'Acc_x', 'Acc_y', 'Acc_z']]
          self.df_euler = self.df[['Euler_x', 'Euler_y', 'Euler_z']]
          self.df_quat = self.df[['Quat_0', 'Quat_1', 'Quat_2', 'Quat_3']]

          return df

      except Exception as e:
          print("❌ An error occurred during data fetching:")
          print(f"Error details: {e}")
          print(f"URL attempted: {dataset_url}")
          return None

  def plot(self, df, title="", show_points=True):
    """
    Plot DataFrame columns in a single line chart with data points
    Parameters:
    -----------
    df : pandas DataFrame
        The dataframe containing data to plot
    title : str, optional
        Title of the plot (default: "All Data Plot")
    show_points : bool, optional
        Whether to show data points on lines (default: True)
    """

    # Create a copy with index as column for x-axis
    df_plot = df.reset_index()

    # Create the line plot
    fig = px.line(df_plot,
                  x='index',
                  y=df.columns,
                  title=title,
                  labels={'index': 'Sample Index', 'value': 'Value'},
                  # template='plotly_dark'
                  )
    if show_points:
        for trace in fig.data:
            trace.update(mode='lines+markers', marker=dict(size=4))

    fig.update_layout(
        legend_title_text='Columns',
        hovermode='x unified',
        xaxis_title='Sample Index',
        yaxis_title='Value',
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=1.02
        )
    )

    # fig.show()
    return fig


def accel_to_angles(df):
    """
    Compute roll and pitch from accelerometer assuming gravity is dominant.
    Returns (phi_acc, theta_acc) in radians.
    """
    ax = df['Acc_x']
    ay = df['Acc_y']
    az = df['Acc_z']

    phi_acc = np.arctan2(ay, az)  # roll
    theta_acc = np.arctan2(-ax, np.sqrt(ay**2 + az**2))  # pitch

    z = np.array([[phi_acc],
                  [theta_acc]], dtype=float)
    return z

def wrap_angle(angle):
    """
    Wrap angle to [-pi, pi] for numerical stability (optional but nice).
    """
    return (angle + np.pi) % (2 * np.pi) - np.pi


## Initialization

Selected Dataset
https://www.kaggle.com/datasets/banaankiamanesh/imu-for-ai

he Data Contains of 3-axis GYRO + 3-axis Accelerometer + 3-axis Magnetometer data, along with the labels(containing pitch, roll, yaw and quaternions). So any supervised multivariate regression approach can be implemented on it.

Sampling frequency of the data is EXACTLY equal to 100 Hz. IMU used for the data collection procedure is: BNO055 9-axis IMU by BOSCH.

In [ ]:
# Initialize
url = 'https://raw.githubusercontent.com/janithcyapa/learn-control/refs/heads/main/Kalman%20Filter/IMU_Data.csv'
utils = utilities(url,0,1000)

✅ Data loading successful!
Total number of data rows loaded: 30000
Total number of data rows selected: 1000

Head of the loaded DataFrame:
   Gyro_x  Gyro_y  Gyro_z  Acc_x  Acc_y  Acc_z  Euler_x  Euler_y  Euler_z  \
0 -0.6250 -0.3125 -0.3125  -0.48   0.09   9.40    -0.75  -3.4375     4.25   
1  0.3750  0.0000  0.3125  -0.51   0.13   9.50    -0.75  -3.4375     4.25   
2  0.6250 -0.0625  0.5625  -0.62   0.22   9.56    -0.75  -3.4375     4.25   
3  0.3125 -0.5000  0.0625  -0.65   0.27   9.58    -0.75  -3.4375     4.25   
4 -0.3125 -1.0625 -0.2500  -0.55   0.22   9.55    -0.75  -3.4375     4.25   

    Quat_0    Quat_1    Quat_2   Quat_3  
0 -0.99884 -0.007874 -0.029846  0.03717  
1 -0.99884 -0.007874 -0.029846  0.03717  
2 -0.99884 -0.007935 -0.029846  0.03717  
3 -0.99884 -0.007935 -0.029846  0.03717  
4 -0.99884 -0.007935 -0.029724  0.03717  


In [ ]:
utils.plot(utils.df_acc, title="Accelerometer Data")

In [ ]:
utils.plot(utils.df_gyro, title="Gyroscope Data")


In [ ]:
utils.plot(utils.df_euler, title="Euler Angles")


# Basic Theory

The Kalman Filter is a state estimation algorithm that provides both an estimate of the current state and a prediction of the future state, along with a measure of their uncertainty. Moreover, it is an optimal algorithm that minimizes state estimation uncertainty. That is why the Kalman Filter has become such a widely used and trusted algorithm.

 The Kalman Filter output is a multivariate random variable. A covariance matrix describes the squared uncertainty of the multivariate random variable.

The uncertainty variables of the multivariate Kalman Filter are:

- $P_{n,n}$ - is a covariance matrix that describes the squared uncertainty of an estimate
- $P_{n+1,n}$ - is a covariance matrix that describes the squared uncertainty of a prediction
- $R_n$ - is a covariance matrix that describes the squared measurement uncertainty
- $Q$ - is a covariance matrix that describes the process noise

### State Extrapolation Equation

Using the state extrapolation equation, we can predict the next system state based on the knowledge of the current state. It extrapolates the state vector from the present (time step n) to the future (time step n+1).

The predicted system state equation is:
$$\hat{x}_{n+1,n} = F\hat{x}_{n,n} + Gu_n + w_n$$

Where:
* $\hat{x}_{n+1,n}$ is a **predicted system state vector** at time step $n + 1$
* $\hat{x}_{n,n}$ is an **estimated system state vector** at time step $n$
* $u_n$ is a **control variable** or **input variable** - a $\text{measurable}$ (deterministic) input to the system
* $w_n$ is a **process noise** or disturbance - an $\text{unmeasurable}$ input that affects the state
* $F$ is a **state transition matrix**
* $G$ is a **control matrix** or $\text{input transition matrix}$ (mapping control to state variables)


#### Linear dynamic systems

For zero-order hold sampling, assuming the input is piecewise constant, the general solution of the state space equation in the form of:
$$\dot{x}(t) = Ax(t) + Bu(t)$$
is given by:
$$x(t + \Delta t) = \underbrace{e^{A\Delta t}}_{\text{F}} x(t) + \underbrace{\int_{0}^{\Delta t} e^{A\tau} B u(t) d\tau}_{\text{G}}$$


### Covariance Extrapolation Equation

The general form of the Covariance Extrapolation Equation is given by:

$$P_{n+1,n} = FP_{n,n}F^T + Q$$

Where:
* $P_{n,n}$ is the **squared uncertainty of an estimate** (covariance matrix) of the current state
* $P_{n+1,n}$ is the **squared uncertainty of a prediction** (covariance matrix) for the next state
* $F$ is the **state transition matrix** that we derived in the "Modeling linear dynamic systems" section
* $Q$ is the **process noise matrix**


$w_n$ is the process noise at the time step n. In the multidimensional case, the process noise is a covariance matrix denoted by Q. Process noise variance has a critical influence on the Kalman Filter performance. Too small q causes a lag error. If the q value is too high, the Kalman Filter follows the measurements and produces noisy estimations.

The process noise can be independent between different state variables. In this case, the process noise covariance matrix Q is a diagonal matrix.The process noise can also be dependent. For example, the constant velocity model assumes zero acceleration (a=0). However, a random variance in acceleration causes a variance in velocity and position. In this case, the process noise is correlated with the state variables.

There are two models for the environmental process noise.
- Discrete noise model
- Continuous noise model

### Measurement Equation

The generalized measurement equation in matrix form is given by:

$$z_n = Hx_n + v_n$$

Where:
* $\mathbf{z}_n$ is a **measurement vector**
* $\mathbf{x}_n$ is a **true system state** (hidden state)
* $\mathbf{v}_n$ is a **random noise vector**
* $\mathbf{H}$ is an **observation matrix**

In many cases, the measured value is not the desired system state. For example, a digital electric thermometer measures an electric current, while the system state is the temperature. There is a need for a transformation of the system state (input) to the measurement (output). The purpose of the observation matrix H is to convert the system state into outputs using linear transformations. The following chapters include examples of observation matrix usage.

 ### Covariance Equations

#### Measurement Uncertainty

The measurement covariance is given by:
$$R_n = E(\mathbf{v}_n \mathbf{v}_n^T)$$

Where:
* $\mathbf{R}_n$ is the **covariance matrix of the measurement**
* $\mathbf{v}_n$ is the **measurement error**

#### Process Noise Uncertainty

The process noise covariance is given by:
$$Q_n = E(\mathbf{w}_n \mathbf{w}_n^T)$$

Where:
* $\mathbf{Q}_n$ is the **covariance matrix of the process noise**
* $\mathbf{w}_n$ is the **process noise**

#### Estimation Uncertainty

The estimation covariance is given by:
$$P_{n,n} = E(\mathbf{e}_n \mathbf{e}_n^T) = E((\mathbf{x}_n - \hat{\mathbf{x}}_{n,n}) (\mathbf{x}_n - \hat{\mathbf{x}}_{n,n})^T)$$

Where:
* $\mathbf{P}_{n,n}$ is the **covariance matrix of the estimation error**
* $\mathbf{e}_n$ is the **estimation error**
* $\mathbf{x}_n$ is the **true system state** (hidden state)
* $\hat{\mathbf{x}}_{n,n}$ is the **estimated system state vector at time step $n$**

### State Update Equation

The State Update Equation in the matrix form is given by,

$$\hat{x}_{n,n} = \hat{x}_{n,n-1} + K_n (z_n - H\hat{x}_{n,n-1})$$

Where:
* $\hat{x}_{n,n}$ is an **estimated system state vector at time step $n$**
* $\hat{x}_{n,n-1}$ is a **predicted system state vector at time step $n - 1$**
* $K_n$ is a **Kalman Gain**
* $z_n$ is a **measurement**
* $H$ is an **observation matrix**


### Covariance Update Equation

The Covariance Update Equation is given by,

$$P_{n,n} = (I - K_n H) P_{n,n-1} (I - K_n H)^T + K_n R_n K_n^T$$

where:
* $P_{n,n}$ is the **covariance matrix of the current state estimation**
* $P_{n,n-1}$ is the **prior estimate covariance matrix of the current state** (predicted at the previous state)
* $K_n$ is the **Kalman Gain**
* $H$ is the **observation matrix**
* $R_n$ is the **measurement noise covariance matrix**
* $I$ is an **Identity Matrix** (the $n \times n$ square matrix with ones on the main diagonal and zeros elsewhere)

### Kalman Gain Equation

The Kalman Gain in matrix notation is given by,

$$K_n = P_{n,n-1}H^T (HP_{n,n-1}H^T + R_n)^{-1}$$

Where:
* $\mathbf{K}_n$ is the **Kalman Gain**
* $\mathbf{P}_{n,n-1}$ is the **prior estimate covariance matrix of the current state** (predicted at the previous step)
* $\mathbf{H}$ is the **observation matrix**
* $\mathbf{R}_n$ is the **measurement noise covariance matrix**

## Summary


![](https://kalmanfilter.net/img/summary/KalmanFilterDiagram.png?v=0)
The following table summarizes notation (including differences found in the literature) and dimensions.

| Term | Name | Alternative term | Dimensions |
| :---: | :--- | :---: | :---: |
| $\mathbf{x}$ | State Vector | | $n_x \times 1$ |
| $\mathbf{z}$ | Measurements Vector | $\mathbf{y}$ | $n_z \times 1$ |
| $\mathbf{F}$ | State Transition Matrix | $\mathbf{\Phi}, \mathbf{A}$ | $n_x \times n_x$ |
| $\mathbf{u}$ | Input Variable | | $n_u \times 1$ |
| $\mathbf{G}$ | Control Matrix | $\mathbf{B}$ | $n_x \times n_u$ |
| $\mathbf{P}$ | Estimate Covariance | $\mathbf{\Sigma}$ | $n_x \times n_x$ |
| $\mathbf{Q}$ | Process Noise Covariance | | $n_x \times n_x$ |
| $\mathbf{R}$ | Measurement Covariance | | $n_z \times n_z$ |
| $\mathbf{w}$ | Process Noise Vector | | $n_x \times 1$ |
| $\mathbf{v}$ | Measurement Noise Vector | | $n_z \times 1$ |
| $\mathbf{H}$ | Observation Matrix | $\mathbf{C}$ | $n_z \times n_x$ |
| $\mathbf{K}$ | Kalman Gain | | $n_x \times n_z$ |
| $\mathbf{n}$ | Discrete-Time Index | $\mathbf{k}$ | |

# Linear Euler-Angle Kalman Filter

## Mathematical Model


In this note, I derive a simple **linear Kalman filter** for estimating **Euler angles**,
- Roll, $\phi$
- Pitch, $\theta$
- Yaw, $\psi$

Assume:
- **Gyroscope** gives angular rates $(p, q, r)$ around body axes.
- **Accelerometer** is used to estimate roll and pitch from gravity.
- **Yaw is not corrected** (it will drift).

Use a **discrete-time linear state-space model** and the **standard Kalman filter**,
- Prediction: use gyro to propagate angles.
- Update: use accelerometer to correct roll & pitch.

### 1. State, Inputs, and Measurements


#### 1.1 State Vector
Define the state vector as the three Euler angles,

$
x_k =
\begin{bmatrix}
\phi_k\\
\theta_k\\
\psi_k
\end{bmatrix}
$

where
- $\phi_k$: roll angle at time step k
- $\theta_k$: pitch angle at time step k
- $\psi_k$: yaw angle at time step k


#### 1.2 Input Vector (Gyroscope Angular Rates)
From the gyroscope, we measure the angular rates around the body axes:

$
u_k =
\begin{bmatrix}
p_k\\
q_k\\
r_k
\end{bmatrix}
$

where

- $p_k$: roll rate (about body $x$-axis),
- $q_k$: pitch rate (about body $y$-axis),
- $r_k$: yaw rate (about body $z$-axis).

#### 1.3 Measurement Vector (Accelerometer-Based Angles)

From the accelerometer readings $a_x, a_y, a_z$, assuming that **gravity dominates** (no large linear accelerations), we can estimate roll and pitch:

$
\phi^{acc}_k = \operatorname{atan2}(a_{y,k}, a_{z,k})
$

$
\theta^{acc}_k =
\operatorname{atan2}\left(-a_{x,k}, \sqrt{a_{y,k}^2 + a_{z,k}^2}\right)
$

We do **not** get yaw from the accelerometer in this simple model.

Thus, the measurement vector is:

$
z_k =
\begin{bmatrix}
\phi^{acc}_k\\
\theta^{acc}_k
\end{bmatrix}
$




### 2. Process (Prediction) Model

Let the sampling time be $\Delta t$.

For a very simple linear model (small-angle, low-dynamics approximation), we use:

$
\dot{\phi} \approx p,\quad
\dot{\theta} \approx q,\quad
\dot{\psi} \approx r
$

Discretizing with Euler integration:

$\phi_{k+1} = \phi_k + \Delta t \, p_k + w_{\phi,k}$

$\theta_{k+1} = \theta_k + \Delta t \, q_k + w_{\theta,k}$

$\psi_{k+1} = \psi_k + \Delta t \, r_k + w_{\psi,k}$

In compact matrix form:

$
x_{k+1} = F x_k + B u_k + w_k
$

with

$
F = I_3 =
\begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
0 & 0 & 1
\end{bmatrix}
$

$
B = \Delta t \, I_3 =
\Delta t
\begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
0 & 0 & 1
\end{bmatrix}
$

The process noise $w_k$ models gyro noise, bias, and unmodeled dynamics:

$w_k \sim \mathcal{N}(0, Q)$

where $Q$ is the **process noise covariance**:

$
Q =
\begin{bmatrix}
\sigma^2_{\phi,proc} & 0 & 0\\
0 & \sigma^2_{\theta,proc} & 0\\
0 & 0 & \sigma^2_{\psi,proc}
\end{bmatrix}
$

The values $\sigma^2_{\phi,proc}$, $\sigma^2_{\theta,proc}$, and $\sigma^2_{\psi,proc}$ are tuning parameters that represent how much uncertainty we allow in the model (e.g. gyro drift).




### 3. Measurement (Update) Model

We only measure roll and pitch from the accelerometer, not yaw. We assume a **linear measurement model**:

$z_k = H x_k + v_k$

with measurement matrix $H$:

$
H =
\begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0
\end{bmatrix}
$

This simply picks out roll and pitch from the state:

$
z_k =
\begin{bmatrix}
\phi^{acc}_k\\
\theta^{acc}_k
\end{bmatrix}
$

$
z_k =
\begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0
\end{bmatrix}
\begin{bmatrix}
\phi_k\\
\theta_k\\
\psi_k
\end{bmatrix}
+ v_k
$

The measurement noise $v_k$ is assumed Gaussian:

$v_k \sim \mathcal{N}(0, R)$

with measurement noise covariance:

$
R =
\begin{bmatrix}
\sigma^2_{\phi,meas} & 0\\
0 & \sigma^2_{\theta,meas}
\end{bmatrix}
$

where $\sigma^2_{\phi,meas}$ and $\sigma^2_{\theta,meas}$ represent how noisy the accelerometer-based roll and pitch estimates are.

### 4. Kalman Filter Equations


We maintain:

- $\hat{x}_{k|k}$: estimate of $x_k$ after using measurements up to time $k$,
- $P_{k|k}$: covariance matrix of estimation error at time $k$.

The Kalman filter has two main steps at each time $k$:

1. **Prediction** (Time Update),
2. **Correction** (Measurement Update).


#### 4.1. Prediction Step

Given the previous estimate $\hat{x}_{k-1|k-1}$ and covariance $P_{k-1|k-1}$, and the input $u_{k-1}$, we predict:

**Predicted state:**

$
\hat{x}_{k|k-1} = F \hat{x}_{k-1|k-1} + B u_{k-1}
$

In components:

$\hat{\phi}_{k|k-1}   = \hat{\phi}_{k-1|k-1}   + \Delta t\, p_{k-1}$

$\hat{\theta}_{k|k-1} = \hat{\theta}_{k-1|k-1} + \Delta t\, q_{k-1}$

$\hat{\psi}_{k|k-1}   = \hat{\psi}_{k-1|k-1}   + \Delta t\, r_{k-1}$

**Predicted covariance:**

$P_{k|k-1} = F P_{k-1|k-1} F^T + Q$

Since $F = I_3$, this simplifies to:

$P_{k|k-1} = P_{k-1|k-1} + Q$



#### 4.2. Update Step

We now incorporate the measurement $z_k$, derived from the accelerometer:

$
z_k =
\begin{bmatrix}
\phi^{acc}_k\\
\theta^{acc}_k
\end{bmatrix}
$

**Innovation (measurement residual):**

$
y_k = z_k - H \hat{x}_{k|k-1}
$

Explicitly,

$
y_k =
\begin{bmatrix}
\phi^{acc}_k\\
\theta^{acc}_k
\end{bmatrix}
-
\begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0
\end{bmatrix}
\begin{bmatrix}
\hat{\phi}_{k|k-1}\\
\hat{\theta}_{k|k-1}\\
\hat{\psi}_{k|k-1}
\end{bmatrix}
=
\begin{bmatrix}
\phi^{acc}_k - \hat{\phi}_{k|k-1}\\
\theta^{acc}_k - \hat{\theta}_{k|k-1}
\end{bmatrix}
$

**Innovation covariance:**

$
S_k = H P_{k|k-1} H^T + R
$

Here,

- $P_{k|k-1}$ is $3 \times 3$,
- $H$ is $2 \times 3$,
- $S_k$ is $2 \times 2$.

**Kalman gain:**

$
K_k = P_{k|k-1} H^T S_k^{-1}
$

- $K_k$ is $3 \times 2$, so it updates all three angles using the innovation in roll and pitch.


**Updated state estimate:**

$
\hat{x}_{k|k} = \hat{x}_{k|k-1} + K_k y_k
$

This step **strongly corrects roll and pitch**. Yaw has no direct measurement, so its update is only indirect (through covariance coupling, if present). In this simple setup, yaw is effectively driven by gyro integration and will drift over time.

**Updated covariance:**

$
P_{k|k} = (I_3 - K_k H) P_{k|k-1}
$


In [ ]:
##

## Implemetation

In [135]:
import numpy as np
import pandas as pd

class KF:
    def __init__(self, x0, P0, Q, R, H, F, B, df, dt):
        """
        Kalman Filter for IMU data fusion

        Parameters:
        -----------
        x0 : numpy array
            Initial state [phi, theta, psi]
        P0 : numpy array
            Initial covariance matrix
        Q : numpy array
            Process noise covariance
        R : numpy array
            Measurement noise covariance
        H : numpy array
            Measurement matrix
        F : numpy array
            State transition matrix
        B : numpy array
            Control input matrix
        df : pandas DataFrame
            Input Dataframe with IMU data
        dt : float
            Time step (1/sample frequency)
        """
        self.x = x0   # Initial state
        self.P = P0   # Initial covariance P (FIXED: was P, should be P0)
        self.Q = Q    # Process noise covariance Q
        self.R = R    # Measurement noise covariance R
        self.H = H    # Measurement matrix H
        self.F = F    # State transition matrix
        self.B = B    # Control input matrix

        self.df = df  # Input Dataframe
        self.dt = dt  # Time step
        self.N = len(df)  # Number of samples

        # Initialize results DataFrame
        self.results = pd.DataFrame(index=range(self.N))  # Results Dataframe
        self.results['Est_x'] = np.zeros(self.N)  # phi estimate
        self.results['Est_y'] = np.zeros(self.N)  # theta estimate
        self.results['Est_z'] = np.zeros(self.N)  # psi estimate
        self.results['Mes_x'] = np.zeros(self.N)  # phi from accelerometer
        self.results['Mes_y'] = np.zeros(self.N)  # theta from accelerometer

    def run(self):
        """
        Run the Kalman filter on all data
        """
        # Main Kalman filter loop
        for k in range(self.N):
            # Get current row from DataFrame
            self.df_k = self.df.iloc[k]

            # --- Read sensors ---
            self.u = np.array([[self.df_k['Gyro_x']],
                               [self.df_k['Gyro_y']],
                               [self.df_k['Gyro_z']]], dtype=float)

            # --- Prediction step ---
            self.x = self.F @ self.x + self.B @ self.u  # State prediction: x_{k|k-1} = F x_{k-1|k-1} + B u

            # Optionally wrap angles (uncomment if needed)
            # self.x[0, 0] = wrap_angle(self.x[0, 0])
            # self.x[1, 0] = wrap_angle(self.x[1, 0])
            # self.x[2, 0] = wrap_angle(self.x[2, 0])

            self.P = self.F @ self.P @ self.F.T + self.Q  # Covariance prediction: P_{k|k-1} = F P F^T + Q

            # --- Measurement from accelerometer ---
            self.z = accel_to_angles(self.df_k)

            # --- Update step ---
            self.y = self.z - (self.H @ self.x)  # Innovation: y_k = z_k - H x_{k|k-1}
            self.S = self.H @ self.P @ self.H.T + self.R  # Innovation covariance: S_k = H P H^T + R
            self.K = self.P @ self.H.T @ np.linalg.inv(self.S)  # Kalman gain: K_k = P H^T S^{-1}
            self.x = self.x + self.K @ self.y  # State update: x_{k|k} = x_{k|k-1} + K y_k
            self.P = (np.eye(3) - self.K @ self.H) @ self.P  # Covariance update: P_{k|k} = (I - K H) P_{k|k-1}

            # Store estimates - FIXED: Use .at for scalar assignment and self. variables
            self.results.at[k, 'Est_x'] = np.degrees(self.x[0, 0])  # phi estimate
            self.results.at[k, 'Est_y'] = np.degrees(self.x[1, 0])  # theta estimate
            self.results.at[k, 'Est_z'] = np.degrees(self.x[2, 0])  # psi estimate
            self.results.at[k, 'Mes_x'] = np.degrees(self.z[0, 0])  # phi from accelerometer (measurement)
            self.results.at[k, 'Mes_y'] = np.degrees(self.z[1, 0]) # theta from accelerometer (measurement)

        return self.results



In [141]:
# Measurement matrix H (2x3) - picks out roll and pitch
H = np.array([[1, 0, 0],
              [0, 1, 0]], dtype=float)

I3 = np.eye(3)
# F = I
F = I3
#B = dt * I
B = dt * I3



# Initial state (start assuming 0 angles)
x0 = np.array([[0.0],
              [0.0],
              [0.0]], dtype=float)

# Initial covariance P (start with some uncertainty)
P0 = np.diag([0.1, 0.1, 0.5])  # rad^2

# Constant process noise covariance Q (gyro/process noise)
# These are tuning values: larger -> trust gyro less (allow more drift).
Q = np.diag([
    (0.03)**2,  # roll process noise
    (0.03)**2,  # pitch process noise
    (0.05)**2   # yaw process noise (can be larger, yaw tends to drift more)
])

# Measurement noise covariance R (accel-based roll & pitch noise)
R = np.diag([
    (0.02)**2,  # roll measurement noise
    (0.02)**2   # pitch measurement noise
])

kf = KF(x0,P0,Q,R,H,F,B,utils.df_gyro_acc,dt)
results = kf.run()
display(results)

df_x = pd.DataFrame()
df_x['Euler_x'] = utils.df_euler['Euler_x']
df_x['Measured_x'] = kf.results['Mes_x']
df_x['Estimated_x'] = kf.results['Est_x']

fig_x = utils.plot(df_x,"X Angle")
fig_x.show()

df_y = pd.DataFrame()
df_y['Euler_y'] = utils.df_euler['Euler_y']
df_y['Measured_y'] = kf.results['Mes_y']
df_y['Estimated_y'] = kf.results['Est_y']

fig_y = utils.plot(df_y,"Y Angle")
fig_y.show()

,Est_x,Est_y,Est_z,Mes_x,Mes_y
0,0.544980,2.910820,-0.179049,0.548560,2.923069
1,0.778309,3.034531,0.000000,0.783999,3.072642
2,1.272983,3.532566,0.322289,1.318289,3.709659
3,1.573803,3.721569,0.358099,1.614381,3.880016
4,1.338440,3.249634,0.214859,1.319669,3.295244
...,...,...,...,...,...
995,-0.363903,-9.888678,-7204.973495,20.055430,-34.626005
996,-2.231047,-9.871871,-7023.167111,17.689400,-33.548525
997,-4.033471,-10.688569,-6846.947065,15.920582,-32.780944
998,-5.593331,-12.096586,-6676.599837,14.763770,-32.201571
